# Analysis of ccgcrv fits

### Settings

In [ ]:
from ccgcrv import ccg_filter
from ccgcrv import ccg_dates
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from copy import copy, deepcopy

In [ ]:
# Function to do the ccg fits and return results in a practical format
def ccg_fits(data, pars):
    data = data.loc[data['co2_ppm'].notna(), :] # Data cannot have missing values.
    xp = data["dec_year"].to_numpy()
    yp = data["co2_ppm"].to_numpy()

    # create the ccgfilt object
    filt = ccg_filter.ccgFilter(xp=xp, yp=yp, shortterm=pars['shortterm'],
                                longterm=pars['longterm'], sampleinterval=pars['sampleinterval'],
                                numpolyterms=pars['numpolyterms'], numharmonics=pars['numharmonics'],
                                timezero=pars['timezero'], gap=pars['gap'], debug=pars['debug'])
    # filt = ccg_filter.ccgFilter(xp=xp, yp=yp)

    pred1 = pd.DataFrame({
        'x': xp,
        'func': filt.getFunctionValue(xp),
        'smooth': filt.getSmoothValue(xp),
        'trend': filt.getTrendValue(xp),
        'harm': filt.getHarmonicValue(xp),
        'poly': filt.getPolyValue(xp)
        })
    
    x0 = filt.xinterp
    pred2 = pd.DataFrame({
        'x0': x0,
        'func': filt.getFunctionValue(x0),
        'smooth': filt.getSmoothValue(x0),
        'trend': filt.getTrendValue(x0),
        'harm': filt.getHarmonicValue(x0),
        'poly': filt.getPolyValue(x0)
        })

    # mm = filt.getMonthlyMeans()
    amps = filt.getAmplitudes()
    yearly = pd.DataFrame(amps,
        columns=['year', 'amp', 'max_date', 'max_value', 'min_date', 'min_value']) # Returns a list of tuples, each tuple has 6 values (year, total_amplitude, max_date, max_value, min_date, min_value)

    tcup, tcdown = filt.getTrendCrossingDates()

    decyear = pd.Series(tcdown)
    cal = decyear.transform(ccg_dates.calendarDate)
    tcd = pd.DataFrame(cal.to_list(), 
        columns=['year', 'month', 'day', 'hour', 'minute', 'second'])
    tcd['doy'] = tcd.apply(lambda row: ccg_dates.dayOfYear(row['year'].astype(int),
        row['month'].astype(int), row['day'].astype(int)), axis=1)

    decyear = pd.Series(tcup)
    cal = decyear.transform(ccg_dates.calendarDate)
    tcu = pd.DataFrame(cal.to_list(),
        columns=['year', 'month', 'day', 'hour', 'minute', 'second'])
    tcu['doy'] = tcu.apply(lambda row: ccg_dates.dayOfYear(row['year'].astype(int),
        row['month'].astype(int), row['day'].astype(int)), axis=1)
    
    yearly['tcd_doy'] = tcd['doy']
    yearly['tcd_dec'] = pd.Series(tcdown)
    yearly['tcu_doy'] = tcu['doy']
    yearly['tcu_dec'] = pd.Series(tcup)

    out = {'pred_xobs':pred1, 'pred_xint':pred2, 'yearly':yearly}

    return(out)

In [ ]:
# Function to plot a regression line
def plot_linreg(x_in, y_in, col='grey', fmt='o', lab = '', alpha=0.7):
    from sklearn.linear_model import LinearRegression
    # import statsmodels.api as sma
    model = LinearRegression()  
    x = x_in.values.reshape(-1, 1) # sklearn fit requires a two dimensional array
    y = y_in.values.reshape(-1, 1)
    model.fit(x, y)
    y_pred = model.predict(x)
    r_sq = ": r_sq = {:.3f}".format(model.score(x, y))
    plt.plot(x, y, fmt, color=col, alpha=alpha,label=lab+r_sq)
    plt.plot(x, y_pred, '--', color=col)

In [ ]:
# Function to create plots. Maybe not being used?
def makeplots(ccg_fit):
    
    # Set plot size, etc
    plt.rcParams['figure.figsize'] = [15, 5]

    # Define some colors
    col_0 = 'aqua'
    col_1 = 'teal'
    col_2 = 'orchid'
    col_3 = 'indigo'

    # Plot data
    yrly = ccg_fit['yearly']
    plt.plot(yrly['year'], yrly['amp'], '-')
    plt.show()
    plt.plot(yrly['year'], yrly['tcd_doy']-180, '-', color=col_1, label='downward zero crossing (relative to DOY 180)')
    plt.plot(yrly['year'], yrly['tcu_doy']-320, '-', color=col_2, label='upward zero crossing (relative to DOY 320)')
    plt.legend(loc='upper right')
    plt.show()

### Read in data

In [ ]:
# Read in Joyce data
joyce_yrly = pd.read_csv('../data/ccgcrv_testing/BRW_SZC_JoyceGRL2021.csv')

joyce_monthly = pd.read_csv('../data/ccgcrv_testing/BRW_CO2_monthly_joyce_varyall.csv')
joyce_monthly['dec_year'] = np.round(joyce_monthly.apply(lambda row: ccg_dates.decimalDate(row['year'].astype(int),
        row['month'].astype(int), 1), axis=1), 2) + 0.08 # Adding 0.08 so it ends in 2013.0. Otherwise 2012 amp is not calculated.

joyce_daily = pd.read_csv('../data/ccgcrv_testing/BRW_CO2_daily_joyce_varyall.csv')
joyce_daily['dec_year'] = np.round(joyce_daily.apply(lambda row: ccg_dates.decimalDate(row['year'].astype(int),
        row['month'].astype(int), row['day'].astype(int)), axis=1), 2)

joyce_hourly = pd.read_csv('../data/ccgcrv_testing/BRW_CO2_3hourly_joyce_constmet.csv')
joyce_hourly['dec_year'] = np.round(joyce_hourly.apply(lambda row: ccg_dates.decimalDate(row['year'].astype(int),
        row['month'].astype(int), row['day'].astype(int), row['hour'].astype(int)), axis=1), 2)


### Get ccg fits

In [ ]:
# Default parameter values as used in the ccgcrv code.
ccg_pars_def = {
    'shortterm': 80,
    'longterm': 667,
    'sampleinterval': 0,
    'numpolyterms': 3,
    'numharmonics': 4,
    'timezero': -1,
    'gap': 0,
    'use_gain_factor': False,
    'debug': False
}

# Test parameter sets

# Set sampleinterval to 1
ccg_pars1 = deepcopy(ccg_pars_def)
ccg_pars1['sampleinterval'] = 1


In [ ]:
# Start and end years for analysis: select years to coincide with Joyce et al.
start_y = 1982.0
end_y = 2013.0

# Pt Barrow - In-Situ - Daily
filein = '../data/ccgcrv_testing/brw-insitu_day.txt'
data = pd.read_csv(filein, delimiter = ' ', header = 0, names = ['dec_year', 'co2_ppm'])
data = data.loc[data['dec_year'].between(start_y, end_y), :] 
ccgfit_defpars_brw_daily_insitu = ccg_fits(data=data, pars=ccg_pars_def)

# Pt Barrow - In-Situ - Daily - Pars 1
filein = '../data/ccgcrv_testing/brw-insitu_day.txt'
data = pd.read_csv(filein, delimiter = ' ', header = 0, names = ['dec_year', 'co2_ppm'])
data = data.loc[data['dec_year'].between(start_y, end_y), :] 
ccgfit_pars1_brw_daily_insitu = ccg_fits(data=data, pars=ccg_pars1)

# Pt Barrow - In-Situ - Monthly
filein = '../data/ccgcrv_testing/brw-insitu_mon.txt'
data = pd.read_csv(filein, delimiter = ' ', header = 0, names = ['dec_year', 'co2_ppm'])
data = data.loc[data['dec_year'].between(start_y, end_y), :] 
ccgfit_defpars_brw_monthly_insitu = ccg_fits(data=data, pars=ccg_pars_def)

# Pt Barrow - In-Situ - Monthly - Pars 1
filein = '../data/ccgcrv_testing/brw-insitu_mon.txt'
data = pd.read_csv(filein, delimiter = ' ', header = 0, names = ['dec_year', 'co2_ppm'])
data = data.loc[data['dec_year'].between(start_y, end_y), :] 
ccgfit_pars1_brw_monthly_insitu = ccg_fits(data=data, pars=ccg_pars1)

# Pt Barrow - Flask - Monthly - Pars 1
filein = '../data/ccgcrv_testing/brw-flask_mon.txt'
data = pd.read_csv(filein, delimiter = ' ', header = 0, names = ['dec_year', 'co2_ppm'])
data = data.loc[data['dec_year'].between(start_y, end_y), :] 
ccgfit_pars1_brw_monthly_flask = ccg_fits(data=data, pars=ccg_pars1)

# Pt Barrow - TOMCAT (Joyce) - Monthly - Pars 1
data = joyce_monthly
data = data.loc[data['dec_year'].between(start_y, end_y), :] 
ccgfit_pars1_brw_monthly_tomcat = ccg_fits(data=data, pars=ccg_pars1)

# Pt Barrow - TOMCAT (Joyce) - Daily - Pars 1
data = joyce_daily
data = data.loc[data['dec_year'].between(start_y, end_y), :] 
ccgfit_pars1_brw_daily_tomcat = ccg_fits(data=data, pars=ccg_pars1)

# Pt Barrow - TOMCAT (Joyce) - Hourly - Pars 1
data = joyce_hourly
data = data.loc[data['dec_year'].between(start_y, end_y), :] 
ccgfit_pars1_brw_hourly_tomcat = ccg_fits(data=data, pars=ccg_pars1)


### Comparisons

In [ ]:
# Comparisons plots of amplitudes and ZC

# Define some colors
col_0 = 'aqua'
col_1 = 'teal'
col_2 = 'orchid'
col_3 = 'indigo'
col_4 = 'blue'
col_5 = 'yellowgreen'
col_6 = 'orange'
col_7 = 'olive'
col_8 = 'khaki'

**Check effect of setting sampleinterval=1 (pars1) and daily vs monthly**

In [ ]:

dp_brw_mon_is = ccgfit_defpars_brw_monthly_insitu['yearly']
p1_brw_mon_is = ccgfit_pars1_brw_monthly_insitu['yearly']
dp_brw_dai_is = ccgfit_defpars_brw_daily_insitu['yearly']
p1_brw_dai_is = ccgfit_pars1_brw_daily_insitu['yearly']

plt.rcParams['figure.figsize'] = [12, 5]

plt.plot(dp_brw_dai_is['year'], dp_brw_dai_is['amp'], '-', color=col_1, label='insitu daily ccgpars:def')
plt.plot(p1_brw_dai_is['year'], p1_brw_dai_is['amp'], '--', color=col_2, label='insitu daily ccgpars:sampleinterval=1')
plt.plot(dp_brw_mon_is['year'], dp_brw_mon_is['amp'], '-', color=col_3, label='insitu monthly ccgpars:def')
plt.plot(p1_brw_mon_is['year'], p1_brw_mon_is['amp'], '--', color=col_4, label='insitu monthly ccgpars:sampleinterval=1')
plt.title(label='CO2 yearly amplitudes at Pt Barrow')
plt.legend(loc='upper left')
plt.show()

plt.plot(dp_brw_dai_is['year'], dp_brw_dai_is['tcd_doy'], '-', color=col_1, label='insitu daily ccgpars:def')
plt.plot(p1_brw_dai_is['year'], p1_brw_dai_is['tcd_doy'], '--', color=col_2, label='insitu daily ccgpars:sampleinterval=1')
plt.plot(dp_brw_mon_is['year'], dp_brw_mon_is['tcd_doy'], '-', color=col_3, label='insitu monthly ccgpars:def')
plt.plot(p1_brw_mon_is['year'], p1_brw_mon_is['tcd_doy'], '--', color=col_4, label='insitu monthly ccgpars:sampleinterval=1')
# plt.ylim(top=max(dp_brw_mon_is['tcd_doy'])*1.05)
plt.title(label='Spring Zero Crossing (SZC) at Pt Barrow')
plt.legend(loc='upper right')
plt.show()

**Compare monthly vs daily and insitu vs flask**

In [ ]:

# In the following, all cases using sampleinterval=1 

p1_brw_dai_is = ccgfit_pars1_brw_daily_insitu['yearly']
p1_brw_mon_is = ccgfit_pars1_brw_monthly_insitu['yearly']
p1_brw_mon_fl = ccgfit_pars1_brw_monthly_flask['yearly']

plt.rcParams['figure.figsize'] = [12, 5]

# Plot amplitudes
plt.plot(p1_brw_dai_is['year'], p1_brw_dai_is['amp'], '-', color=col_1, label='insitu daily')
plt.plot(p1_brw_mon_is['year'], p1_brw_mon_is['amp'], '-', color=col_4, label='insitu monthly')
plt.plot(p1_brw_mon_fl['year'], p1_brw_mon_fl['amp'], '-', color=col_0, label='flask monthly')
plt.title(label='CO2 yearly amplitudes at Pt Barrow')
plt.legend(loc='upper left')
plt.show()

# Plot minimum values
plt.plot(p1_brw_dai_is['year'], p1_brw_dai_is['min_value'], '-', color=col_1, label='insitu daily')
plt.plot(p1_brw_mon_is['year'], p1_brw_mon_is['min_value'], '-', color=col_4, label='insitu monthly')
plt.plot(p1_brw_mon_fl['year'], p1_brw_mon_fl['min_value'], '-', color=col_0, label='flask monthly')
plt.title(label='CO2 yearly minimum values at Pt Barrow')
plt.legend(loc='lower left')
plt.show()

# Plot spring zero crossing (SZC)
plt.plot(p1_brw_dai_is['year'], p1_brw_dai_is['tcd_doy'], '-', color=col_1, label='insitu daily')
plt.plot(p1_brw_mon_is['year'], p1_brw_mon_is['tcd_doy'], '-', color=col_4, label='insitu monthly')
plt.plot(p1_brw_mon_fl['year'], p1_brw_mon_fl['tcd_doy'], '-', color=col_0, label='flask monthly')
# plt.ylim(top=max(dp_brw_dai_is['tcd_doy'])*1.1)
plt.title(label='Spring Zero Crossing (SZC) at Pt Barrow')
plt.legend(loc='upper right')
plt.show()


**Compare Joyce data (tomcat calculated vs figure)**

In [ ]:

plt.rcParams['figure.figsize'] = [12, 5]

p1_brw_hou_tc = ccgfit_pars1_brw_hourly_tomcat['yearly']
p1_brw_dai_tc = ccgfit_pars1_brw_daily_tomcat['yearly']
p1_brw_mon_tc = ccgfit_pars1_brw_monthly_tomcat['yearly']

# Plot amplitudes
plt.plot(p1_brw_mon_tc['year'], p1_brw_mon_tc['amp'], '-', color=col_5, label='tomcat monthly')
plt.plot(p1_brw_dai_tc['year'], p1_brw_dai_tc['amp'], '-', color=col_6, label='tomcat daily')
plt.plot(p1_brw_hou_tc['year'], p1_brw_hou_tc['amp'], '-', color=col_4, label='tomcat 3hourly')
plt.title(label='CO2 yearly amplitudes at Pt Barrow')
plt.legend(loc='upper left')
plt.show()

# Plot min_value
plt.plot(p1_brw_mon_tc['year'], p1_brw_mon_tc['min_value'], '-', color=col_5, label='tomcat monthly')
plt.plot(p1_brw_dai_tc['year'], p1_brw_dai_tc['min_value'], '-', color=col_6, label='tomcat daily')
plt.plot(p1_brw_hou_tc['year'], p1_brw_hou_tc['min_value'], '-', color=col_4, label='tomcat 3hourly')
plt.title(label='CO2 yearly minimum value at Pt Barrow')
plt.legend(loc='lower left')
plt.show()

# Plot spring zero crossing (SZC)
plt.plot(p1_brw_mon_tc['year'], p1_brw_mon_tc['tcd_doy'], '-', color=col_5, label='tomcat monthly')
plt.plot(p1_brw_dai_tc['year'], p1_brw_dai_tc['tcd_doy'], '-', color=col_6, label='tomcat daily')
plt.plot(p1_brw_hou_tc['year'], p1_brw_hou_tc['tcd_doy'], '-', color=col_4, label='tomcat 3hourly')
plt.plot(joyce_yrly['year'], joyce_yrly['szc_tomcat_figS9'], '-', color=col_7, label='tomcat figure')
plt.ylim(top=max(p1_brw_mon_tc['tcd_doy'])*1.05)
plt.title(label='Spring Zero Crossing (SZC) at Pt Barrow')
plt.legend(loc='upper right')
plt.show()

# Plot autumn zero crossing (AZC)
plt.plot(p1_brw_mon_tc['year'], p1_brw_mon_tc['tcu_doy'], '-', color=col_5, label='tomcat monthly')
plt.plot(p1_brw_dai_tc['year'], p1_brw_dai_tc['tcu_doy'], '-', color=col_6, label='tomcat daily')
plt.plot(p1_brw_hou_tc['year'], p1_brw_hou_tc['tcu_doy'], '-', color=col_4, label='tomcat 3hourly')
plt.ylim(top=max(p1_brw_mon_tc['tcu_doy'])*1.03)
plt.title(label='Autumn Zero Crossing (AZC) at Pt Barrow')
plt.legend(loc='upper right')
plt.show()

# Plot AZC - SZC (growing season)
plt.plot(p1_brw_mon_tc['year'], p1_brw_mon_tc['tcu_doy'] - p1_brw_mon_tc['tcd_doy'], '-', color=col_5, label='tomcat monthly')
plt.plot(p1_brw_dai_tc['year'], p1_brw_dai_tc['tcu_doy'] - p1_brw_dai_tc['tcd_doy'], '-', color=col_6, label='tomcat daily')
plt.plot(p1_brw_hou_tc['year'], p1_brw_hou_tc['tcu_doy'] - p1_brw_hou_tc['tcd_doy'], '-', color=col_4, label='tomcat 3hourly')
# plt.ylim(top=max(dp_brw_dai_is['tcu_doy'])*1.05)
plt.title(label='Growing Season (AZC-SZC) at Pt Barrow')
plt.legend(loc='upper right')
plt.show()

# Plot tomcat from figure vs tomcat calculated
plt.rcParams['figure.figsize'] = [5, 5]
xy = pd.DataFrame({'x': joyce_yrly['szc_tomcat_figS9'], 'y': p1_brw_dai_tc['tcd_doy']})
xy = xy.sort_values(by='x')
plot_linreg(xy['x'], xy['y'], fmt='o', col=col_1, lab='insitu daily', alpha=1)
plt.title(label='SZC at Pt Barrow: tomcat paper figure vs calculated')
plt.legend(loc='upper left')
plt.show()


**Compare observed vs tomcat (Joyce data)**

In [ ]:


plt.rcParams['figure.figsize'] = [12, 5]

p1_brw_dai_is = ccgfit_pars1_brw_daily_insitu['yearly']
p1_brw_mon_is = ccgfit_pars1_brw_monthly_insitu['yearly']
p1_brw_dai_tc = ccgfit_pars1_brw_daily_tomcat['yearly']
p1_brw_mon_tc = ccgfit_pars1_brw_monthly_tomcat['yearly']

# Plot amplitudes
plt.plot(p1_brw_dai_is['year'], p1_brw_dai_is['amp'], '-', color=col_1, label='insitu daily')
plt.plot(p1_brw_mon_is['year'], p1_brw_mon_is['amp'], '-', color=col_4, label='insitu monthly')
plt.plot(p1_brw_dai_tc['year'], p1_brw_dai_tc['amp'], '-', color=col_6, label='tomcat daily')
plt.plot(p1_brw_mon_tc['year'], p1_brw_mon_tc['amp'], '-', color=col_5, label='tomcat monthly')
plt.title(label='CO2 yearly amplitudes at Pt Barrow')
plt.legend(loc='upper left')
plt.show()

# Plot minimum value
plt.plot(p1_brw_dai_is['year'], p1_brw_dai_is['min_value'], '-', color=col_1, label='insitu daily')
plt.plot(p1_brw_mon_is['year'], p1_brw_mon_is['min_value'], '-', color=col_4, label='insitu monthly')
plt.plot(p1_brw_dai_tc['year'], p1_brw_dai_tc['min_value'], '-', color=col_6, label='tomcat daily')
plt.plot(p1_brw_mon_tc['year'], p1_brw_mon_tc['min_value'], '-', color=col_5, label='tomcat monthly')
plt.title(label='CO2 yearly minimum value at Pt Barrow')
plt.legend(loc='lower left')
plt.show()

# Plot spring zero crossing (SZC)
plt.plot(p1_brw_dai_is['year'], p1_brw_dai_is['tcd_doy'], '-', color=col_1, label='insitu daily')
plt.plot(p1_brw_mon_is['year'], p1_brw_mon_is['tcd_doy'], '-', color=col_4, label='insitu monthly')
plt.plot(p1_brw_dai_tc['year'], p1_brw_dai_tc['tcd_doy'], '-', color=col_6, label='tomcat daily')
plt.plot(p1_brw_mon_tc['year'], p1_brw_mon_tc['tcd_doy'], '-', color=col_5, label='tomcat monthly')
plt.plot(joyce_yrly['year'], joyce_yrly['szc_tomcat_figS9'], '-', color=col_7, label='tomcat figure')
plt.ylim(top=max(p1_brw_dai_is['tcd_doy'])*1.05)
plt.title(label='Spring Zero Crossing (SZC) at Pt Barrow')
plt.legend(loc='upper right')
plt.show()

# Plot autumn zero crossing (AZC)
plt.plot(p1_brw_dai_is['year'], p1_brw_dai_is['tcu_doy'], '-', color=col_1, label='insitu daily')
plt.plot(p1_brw_mon_is['year'], p1_brw_mon_is['tcu_doy'], '-', color=col_4, label='insitu monthly')
plt.plot(p1_brw_dai_tc['year'], p1_brw_dai_tc['tcu_doy'], '-', color=col_6, label='tomcat daily')
plt.plot(p1_brw_mon_tc['year'], p1_brw_mon_tc['tcu_doy'], '-', color=col_5, label='tomcat monthly')

# plt.ylim(top=max(p1_brw_dai_is['tcu_doy'])*1.05)
plt.title(label='Autumn Zero Crossing (AZC) at Pt Barrow')
plt.legend(loc='upper right')
plt.show()

# Plot AZC - SZC (growing season)
plot_linreg(p1_brw_dai_is['year'], p1_brw_dai_is['tcu_doy'] - p1_brw_dai_is['tcd_doy'], fmt='-', col=col_1, lab='insitu daily', alpha=1)
plot_linreg(p1_brw_mon_is['year'], p1_brw_mon_is['tcu_doy'] - p1_brw_mon_is['tcd_doy'], fmt='-', col=col_4, lab='insitu monthly', alpha=1)
plot_linreg(p1_brw_dai_tc['year'], p1_brw_dai_tc['tcu_doy'] - p1_brw_dai_tc['tcd_doy'], fmt='-', col=col_6, lab='tomcat daily', alpha=1)
plot_linreg(p1_brw_mon_tc['year'], p1_brw_mon_tc['tcu_doy'] - p1_brw_mon_tc['tcd_doy'], fmt='-', col=col_5, lab='tomcat monthly', alpha=1)
plt.ylim(top=max(p1_brw_dai_tc['tcu_doy'] - p1_brw_dai_tc['tcd_doy'])*1.05)
plt.title(label='Growing Season (AZC-SZC) at Pt Barrow')
plt.legend(loc='upper right')
plt.show()

# Plot tomcat vs observed SZC
plt.rcParams['figure.figsize'] = [5, 5]

xy = pd.DataFrame({'x': joyce_yrly['szc_tomcat_figS9'], 'y': p1_brw_dai_is['tcd_doy']})
xy = xy.sort_values(by='x')
plot_linreg(xy['x'], xy['y'], fmt='o', col=col_3, lab='tc-fig vs obs-daily-is', alpha=0.7)

xy = pd.DataFrame({'x': p1_brw_dai_tc['tcd_doy'], 'y': p1_brw_dai_is['tcd_doy']})
xy = xy.sort_values(by='x')
plot_linreg(xy['x'], xy['y'], fmt='o', col=col_4, lab='tc-daily-calc vs obs-daily-is', alpha=0.7)

plt.title(label='SZC at Pt Barrow: tomcat model vs observed')
plt.legend(loc='upper left')
plt.show()

# Plot tomcat vs observed AZC
plt.rcParams['figure.figsize'] = [5, 5]

xy = pd.DataFrame({'x': p1_brw_dai_tc['tcu_doy'], 'y': p1_brw_dai_is['tcu_doy']})
xy = xy.sort_values(by='x')
plot_linreg(xy['x'], xy['y'], fmt='o', col=col_3, lab='tc-daily-calc vs obs-daily-is', alpha=0.7)

xy = pd.DataFrame({'x': p1_brw_mon_tc['tcu_doy'], 'y': p1_brw_mon_is['tcu_doy']})
xy = xy.sort_values(by='x')
plot_linreg(xy['x'], xy['y'], fmt='o', col=col_4, lab='tc-monthly-calc vs obs-monthly-is', alpha=0.7)

plt.title(label='AZC at Pt Barrow: tomcat model vs observed')
plt.legend(loc='upper left')
plt.show()


### Sensitivity Analysis

**Check sensitivity to shortterm and longterm filters**

In [ ]:

# Default parameter values as used in the ccgcrv code.
ccg_pars_def = {
    'shortterm': 80,
    'longterm': 667,
    'sampleinterval': 0,
    'numpolyterms': 3,
    'numharmonics': 4,
    'timezero': -1,
    'gap': 0,
    'use_gain_factor': False,
    'debug': False
}

# Set sampleinterval to 1
ccg_pars1 = deepcopy(ccg_pars_def)
ccg_pars1['sampleinterval'] = 1

In [ ]:
# Define test parameter sets
mod = (1.2, 1.1, 1, 0.9, 0.8)

# Color list for plotting
cols = [col_0, col_1, col_2, col_3, col_4]

In [ ]:
# Choose data for sensitivity analysis

# filein = '../data/ccgcrv_testing/brw-insitu_day.txt' # pt barrow insitu daily data
filein = '../data/ccgcrv_testing/brw-insitu_mon.txt' # pt barrow insitu monthly data

data1 = pd.read_csv(filein, delimiter = ' ', header = 0, names = ['dec_year', 'co2_ppm'])
data1 = data1.loc[data1['dec_year'].between(start_y, end_y), :] 

In [ ]:
# Plot shortterm amplitude sens
ccg_pars2 = deepcopy(ccg_pars1)
for i in range(len(mod)):
    ccg_pars2['shortterm'] = ccg_pars1['shortterm'] * mod[i]
    ccgfit = ccg_fits(data=data1, pars=ccg_pars2)
    yearly = ccgfit['yearly']

    plt.rcParams['figure.figsize'] = [12, 5]
    plt.plot(yearly['year'], yearly['amp'], '-', color=cols[i], label='shortterm*'+str(mod[i]))
plt.title(label='CO2 amplitudes at Pt Barrow: short term filter sensitivity')
plt.legend(loc='upper left')
plt.show()

# Plot shortterm SZC sens
ccg_pars2 = deepcopy(ccg_pars1)
for i in range(len(mod)):
    ccg_pars2['shortterm'] = ccg_pars1['shortterm'] * mod[i]
    ccgfit = ccg_fits(data=data1, pars=ccg_pars2)
    yearly = ccgfit['yearly']

    plt.rcParams['figure.figsize'] = [12, 5]
    plt.plot(yearly['year'], yearly['tcd_doy'], '-', color=cols[i], label='shortterm*'+str(mod[i]))
plt.title(label='CO2 SZC at Pt Barrow: short term filter sensitivity')
plt.legend(loc='lower left')
plt.show()

# Plot longterm AZC-SZC sens
ccg_pars2 = deepcopy(ccg_pars1)
for i in range(len(mod)):
    ccg_pars2['shortterm'] = ccg_pars1['shortterm'] * mod[i]
    ccgfit = ccg_fits(data=data1, pars=ccg_pars2)
    yearly = ccgfit['yearly']

    plt.rcParams['figure.figsize'] = [12, 5]
    plt.plot(yearly['year'], yearly['tcu_doy']-yearly['tcd_doy'], '-', color=cols[i], label='shortterm*'+str(mod[i]))
plt.title(label='CO2 AZC-SZC at Pt Barrow: short term filter sensitivity')
plt.legend(loc='lower right')
plt.show()

In [ ]:

# Plot longterm amplitude sens
ccg_pars2 = deepcopy(ccg_pars1)
for i in range(len(mod)):
    ccg_pars2['longterm'] = ccg_pars1['longterm'] * mod[i]
    ccgfit = ccg_fits(data=data1, pars=ccg_pars2)
    yearly = ccgfit['yearly']

    plt.rcParams['figure.figsize'] = [12, 5]
    plt.plot(yearly['year'], yearly['amp'], '-', color=cols[i], label='longterm*'+str(mod[i]))
plt.title(label='CO2 amplitudes at Pt Barrow: long term filter sensitivity')
plt.legend(loc='upper left')
plt.show()

# Plot longterm SZC sens
ccg_pars2 = deepcopy(ccg_pars1)
for i in range(len(mod)):
    ccg_pars2['longterm'] = ccg_pars1['longterm'] * mod[i]
    ccgfit = ccg_fits(data=data1, pars=ccg_pars2)
    yearly = ccgfit['yearly']

    plt.rcParams['figure.figsize'] = [12, 5]
    plt.plot(yearly['year'], yearly['tcd_doy'], '-', color=cols[i], label='longterm*'+str(mod[i]))
plt.title(label='CO2 SZC at Pt Barrow: long term filter sensitivity')
plt.legend(loc='lower left')
plt.show()

# Plot longterm AZC-SZC sens
ccg_pars2 = deepcopy(ccg_pars1)
for i in range(len(mod)):
    ccg_pars2['longterm'] = ccg_pars1['longterm'] * mod[i]
    ccgfit = ccg_fits(data=data1, pars=ccg_pars2)
    yearly = ccgfit['yearly']

    plt.rcParams['figure.figsize'] = [12, 5]
    plt.plot(yearly['year'], yearly['tcu_doy']-yearly['tcd_doy'], '-', color=cols[i], label='longterm*'+str(mod[i]))
plt.title(label='CO2 AZC-SZC at Pt Barrow: long term filter sensitivity')
plt.legend(loc='lower right')
plt.show()


In [ ]:
# Checking effect of shortterm change on tomcat calculated vs FigS9 fit 

data2 = joyce_daily
data2 = data2.loc[data2['dec_year'].between(start_y, end_y), :] 

# Plot tomcat from figure vs tomcat calculated sens
ccg_pars2 = deepcopy(ccg_pars1)
for i in range(len(mod)):
    ccg_pars2['shortterm'] = ccg_pars1['shortterm'] * mod[i]
    ccgfit = ccg_fits(data=data2, pars=ccg_pars2)
    yearly = ccgfit['yearly']

    plt.rcParams['figure.figsize'] = [5, 5]
    xy = pd.DataFrame({'x': joyce_yrly['szc_tomcat_figS9'], 'y': yearly['tcd_doy']})
    xy = xy.sort_values(by='x')
    plot_linreg(xy['x'], xy['y'], fmt='o', lab='shortterm*'+str(mod[i]), col=cols[i], alpha=0.6)

plt.title(label='BRW SZC tomcat paper figure vs calculated - shortterm sensitivity')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Checking effect of longterm change on tomcat calculated vs FigS9 fit 

data2 = joyce_daily
data2 = data2.loc[data2['dec_year'].between(start_y, end_y), :] 

# Plot tomcat from figure vs tomcat calculated sens
ccg_pars2 = deepcopy(ccg_pars1)
for i in range(len(mod)):
    ccg_pars2['longterm'] = ccg_pars1['longterm'] * mod[i]
    ccgfit = ccg_fits(data=data2, pars=ccg_pars2)
    yearly = ccgfit['yearly']

    plt.rcParams['figure.figsize'] = [5, 5]
    xy = pd.DataFrame({'x': joyce_yrly['szc_tomcat_figS9'], 'y': yearly['tcd_doy']})
    xy = xy.sort_values(by='x')
    plot_linreg(xy['x'], xy['y'], fmt='o', lab='longterm*'+str(mod[i]), col=cols[i], alpha=0.6)

plt.title(label='BRW SZC tomcat paper figure vs calculated - longterm sensitivity')
plt.legend(loc='upper left')
plt.show()

**Checking sensitivity to polinomial fit**

In [ ]:
npolyvals = [2,3,4,5]
cols = [col_1, col_2, col_3, col_4]

# Plot numpolyterms amplitude sens
ccg_pars2 = deepcopy(ccg_pars1)
for i in range(len(npolyvals)):
    ccg_pars2['numpolyterms'] = npolyvals[i]
    ccgfit = ccg_fits(data=data1, pars=ccg_pars2)
    yearly = ccgfit['yearly']

    plt.rcParams['figure.figsize'] = [12, 5]
    plt.plot(yearly['year'], yearly['amp'], '-', color=cols[i], label='numpolyterms='+str(npolyvals[i]))
plt.title(label='CO2 amplitudes at Pt Barrow: polynomial terms sensitivity')
plt.legend(loc='upper left')
plt.show()

# Plot numpolyterms SZC sens
ccg_pars2 = deepcopy(ccg_pars1)
for i in range(len(npolyvals)):
    ccg_pars2['numpolyterms'] = npolyvals[i]
    ccgfit = ccg_fits(data=data1, pars=ccg_pars2)
    yearly = ccgfit['yearly']

    plt.rcParams['figure.figsize'] = [12, 5]
    plt.plot(yearly['year'], yearly['tcd_doy'], '-', color=cols[i], label='numpolyterms='+str(npolyvals[i]))
plt.title(label='CO2 SZC at Pt Barrow: polynomial terms sensitivity')
plt.legend(loc='lower left')
plt.show()

# Plot numpolyterms AZC-SZC sens
ccg_pars2 = deepcopy(ccg_pars1)
for i in range(len(npolyvals)):
    ccg_pars2['numpolyterms'] = npolyvals[i]
    ccgfit = ccg_fits(data=data1, pars=ccg_pars2)
    yearly = ccgfit['yearly']

    plt.rcParams['figure.figsize'] = [12, 5]
    plt.plot(yearly['year'], yearly['tcu_doy']-yearly['tcd_doy'], '-', color=cols[i], label='numpolyterms='+str(npolyvals[i]))
plt.title(label='CO2 AZC-SZC at Pt Barrow: polynomial terms sensitivity')
plt.legend(loc='lower right')
plt.show()


In [ ]:
npolyvals = [2,3,4,5]
cols = [col_1, col_2, col_3, col_4]

# Plot numpolyterms trends sens
ccg_pars2 = deepcopy(ccg_pars1)
for i in range(len(npolyvals)):
    ccg_pars2['numpolyterms'] = npolyvals[i]
    ccgfit = ccg_fits(data=data1, pars=ccg_pars2)
    pred = ccgfit['pred_xint']

    plt.rcParams['figure.figsize'] = [12, 5]
    plt.plot(pred['x0'], pred['trend'], '-', color=cols[i], label='numpolyterms='+str(npolyvals[i]))
plt.title(label='CO2 trend at Pt Barrow: polynomial terms sensitivity')
plt.legend(loc='upper left')
plt.show()

# Plot numpolyterms trends sens
ccg_pars2 = deepcopy(ccg_pars1)
for i in range(len(npolyvals)):
    ccg_pars2['numpolyterms'] = npolyvals[i]
    ccgfit = ccg_fits(data=data1, pars=ccg_pars2)
    pred = ccgfit['pred_xint']

    plt.rcParams['figure.figsize'] = [12, 5]
    plt.plot(pred['x0'], pred['poly'], '-', color=cols[i], label='numpolyterms='+str(npolyvals[i]))
plt.title(label='CO2 polynomial fit at Pt Barrow: polynomial terms sensitivity')
plt.legend(loc='upper left')
plt.show()
